In [1]:
import pickle
import pandas as pd
from gensim.models import Word2Vec

C:\Users\olive\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
train_df = pd.read_csv("./datasets/cleaned_train.csv", encoding='latin-1')

In [3]:
train_df = train_df[['Sentiment', 'SentimentText']].dropna()

In [4]:
word2vec = Word2Vec.load("./embeddings/fine_tuned_word2vec.model")

C:\Users\olive\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [5]:
def get_word_vectors(df: pd.DataFrame, word2vec):    
    labels = []
    vectors = []
    for idx, row in df.iterrows():
        avgword2vec = None
        count = 0
        for word in row["SentimentText"].split():
            if word in word2vec.wv.vocab:
                count += 1
                if avgword2vec is None:
                    avgword2vec = word2vec[word]
                else:
                    avgword2vec = avgword2vec + word2vec[word]

        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
            labels.append(row["Sentiment"])
            vectors.append(avgword2vec)
            
    representations_df = pd.DataFrame(data={"sentiment":labels, "vector_representation":vectors})
    return representations_df

In [6]:
representations_df = get_word_vectors(train_df.copy(), word2vec)

C:\Users\olive\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
C:\Users\olive\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [7]:
representations_df.head()

,sentiment,vector_representation
0,0,"[0.13012695, -0.083984375, 0.11010742, 0.02478..."
1,0,"[0.09889221, 0.013458252, 0.022888184, 0.00280..."
2,1,"[0.016151832, -0.009662028, 0.019930601, 0.044..."
3,0,"[0.0057941163, 0.024747463, -0.010290599, 0.08..."
4,0,"[-0.13360596, 0.0032958984, 0.09498596, 0.0018..."


In [8]:
pickle.dump(representations_df, open("./datasets/vectors_train_dataset.pickle", "wb"))